In [2]:
# YOLOv8 Training Script
from ultralytics import YOLO

model = YOLO('../src/human_detector/models/yolov11n.pt') # Load a pretrained YOLOv8n model

# Training
#    - data: path to your data.yaml
#    - epochs, imgsz, project, name: same CLI args
results = model.train(
    data    = 'data.yaml',
    epochs  = 1,
    imgsz   = 640,
    project = 'human_robot',
    name    = 'exp1'
)

# `results` is a Python object you can inspect for metrics, best.pt path, etc.
print(f"Trained checkpoint saved to: {results.best}")


New https://pypi.org/project/ultralytics/8.3.142 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.141 🚀 Python-3.10.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce MX550, 1864MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../src/human_detector/models/yolov11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp12, nbs=64, nms=False, opse

100%|██████████| 5.35M/5.35M [00:00<00:00, 14.6MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3124.0±479.0 MB/s, size: 349.9 KB)


train: Scanning /home/yogee/Desktop/human_detector_ws/training_data/labels/train... 1324 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1324/1324 [00:04<00:00, 322.79it/s]

train: New cache created: /home/yogee/Desktop/human_detector_ws/training_data/labels/train.cache


...

val: New cache created: /home/yogee/Desktop/human_detector_ws/training_data/labels/val.cache



/home/yogee/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


Plotting labels to human_robot/exp12/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to human_robot/exp12
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/83 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB. GPU 0 has a total capacity of 1.82 GiB of which 12.00 MiB is free. Including non-PyTorch memory, this process has 1.80 GiB memory in use. Of the allocated memory 1.65 GiB is allocated by PyTorch, and 54.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)